In [ ]:
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openpyxl import load_workbook

from scipy import stats as stats
from copy import copy as copy

import ipywidgets as widgets

In [ ]:
from functions.masterdata import (master_data, readConfig)

In [ ]:
writeOutput = True
# writeOutput = False

autoRunProject = True
# autoRunProject = False


In [ ]:
if autoRunProject:
    try:
        with open('projects.txt', 'r') as f:   ## ToDo: make this robust to lack of file
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if ((not line.startswith('#')) and (not line.strip()=='')):
                    subfolder = line
    except FileNotFoundError:
        subfolder = input("Enter the name of the working folder (Must be same level as code folder)")
        # print('error')
        
else:
    subfolder = input("Enter the name of the working folder (Must be same level as code folder)")

print(subfolder)
os.chdir("../" + subfolder)


In [ ]:
configDict = readConfig()

In [ ]:
dataPath = os.path.join(configDict['rootDir'],configDict['initialDataPath'])
dataPath = os.path.join(configDict['rootDir'],configDict['QCDataPath'])

masterData = master_data(dataPath)


counts, probes, segs = masterData.get_data()


counts

# probes

# segs

In [ ]:
counts.index.name = 'TargetName'

In [ ]:
newIndex = [x for x in counts.index if not x=="NegProbe-WTX"]

In [ ]:
# Remove NegProbe-WTX
counts = counts.loc[newIndex]

In [ ]:
# Export counts to countFile.tsv

In [ ]:
counts.to_csv(os.path.join(configDict['rootDir'], 'countFile.tsv'), sep='\t')


In [ ]:
probes

In [ ]:
featureAnno = probes.loc[:,['TargetName','CodeClass']]

featureAnno['Negative'] = [ 'TRUE' if x == 'Negative' else 'False' for x in featureAnno['CodeClass']]

featureAnno.drop(labels = ['CodeClass'], axis=1, inplace = True)

featureAnno

In [ ]:
featureAnno = featureAnno.loc[newIndex]

In [ ]:
featureAnno['RTS_ID'] = [ 'RTS' + x for x in featureAnno['TargetName']]


featureAnno.reset_index(inplace = True)

featureAnno.set_index('RTS_ID', inplace = True)

In [ ]:
featureAnno

In [ ]:
# Export probe into to featureAnnoFile.tsv

In [ ]:
featureAnno.to_csv(os.path.join(configDict['rootDir'], 'featureAnnoFile.tsv'), sep='\t')

In [ ]:
segs.index.name='SlideName'

In [ ]:
segs.reset_index(drop=True)

In [ ]:
segs.columns

In [ ]:
# Rename Segment Labels to epithelial and Luminal (also cleans data)
segs.loc[segs['SegmentLabel'] == 'Full ROI','SegmentLabel'] = 'Epithelial'
segs.loc[segs['SegmentLabel'] == 'Segment 1','SegmentLabel'] = 'Epithelial'
segs.loc[segs['SegmentLabel'] == 'Segment 2','SegmentLabel'] = 'Luminal'


In [ ]:
segs['SlideName'] = [x.replace(' ','_') for x in segs['SlideName']]
segs['SlideName'] = [x.replace('-','_') for x in segs['SlideName']]
segs['SlideName'] = [x[-7:] for x in segs['SlideName']]

In [ ]:
segs['ScanLabel'] = [x.replace(' ','_') for x in segs['ScanLabel']]
segs['ScanLabel'] = [x.replace('-','_') for x in segs['ScanLabel']]
segs['ScanLabel'] = [x[-7:] for x in segs['ScanLabel']]

In [ ]:
segs.set_index('SlideName', inplace=True)

In [ ]:
# Export segs to sampleAnnoFile.tsv

In [ ]:
segs

In [ ]:
segs.to_csv(os.path.join(configDict['rootDir'], 'sampleAnnoFile.tsv'), sep='\t')
